In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings('ignore')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
insurance = pd.read_csv('insurance.csv', sep=',')

In [4]:
insurance.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [5]:
insurance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [6]:
independente = insurance.iloc[:,0:6].values
dependente = insurance.iloc[:,6].values

In [7]:
dummies = ColumnTransformer(transformers=[('OneHot', OneHotEncoder(), [1,4,5])],
                  remainder='passthrough')

In [8]:
independente = dummies.fit_transform(independente)

In [9]:
independente

array([[1.0, 0.0, 0.0, ..., 19, 27.9, 0],
       [0.0, 1.0, 1.0, ..., 18, 33.77, 1],
       [0.0, 1.0, 1.0, ..., 28, 33.0, 3],
       ...,
       [1.0, 0.0, 1.0, ..., 18, 36.85, 0],
       [1.0, 0.0, 1.0, ..., 21, 25.8, 0],
       [1.0, 0.0, 0.0, ..., 61, 29.07, 0]], dtype=object)

In [10]:
dependente

array([16884.924 ,  1725.5523,  4449.462 , ...,  1629.8335,  2007.945 ,
       29141.3603])

In [11]:
x_treino, x_teste, y_treino, y_teste = train_test_split(independente,dependente,
                                                       test_size=0.3, random_state=10)

In [12]:
x_treino.shape

(936, 11)

In [13]:
x_teste.shape

(402, 11)

In [14]:
y_teste.shape

(402,)

In [15]:
rede = MLPRegressor(hidden_layer_sizes=(80,80,80),activation='relu',solver='adam',
             max_iter=800,tol=0.0001, random_state=10)

#activation = relu,tanh,logistic, identity
#solver = adam,sgd,lbfgs

In [16]:
rede.fit(x_treino,y_treino)

MLPRegressor(hidden_layer_sizes=(80, 80, 80), max_iter=800, random_state=10)

In [17]:
rede.score(x_treino,y_treino)

0.849099752384026

In [18]:
rede.score(x_teste,y_teste)

0.8175508861902884

In [19]:
previsoes = rede.predict(x_teste)

In [20]:
ema = mean_absolute_error(y_teste,previsoes)
mse = mean_squared_error(y_teste,previsoes)
rmse = np.sqrt(mse)

print(f' Média do erro absoluto : {round(ema,2)}')
print(f' Média do erro quadrático : {round(mse,2)}')
print(f' Raiz da média do erro quadrático : {round(rmse,2)}')

 Média do erro absoluto : 2936.69
 Média do erro quadrático : 23692455.48
 Raiz da média do erro quadrático : 4867.49


In [21]:
kfold = KFold(n_splits=15, shuffle=True, random_state=10)
modelo = rede
resultado = cross_val_score(modelo, independente,dependente, cv=kfold)

print(f' Coeficiente de determinação médio : {np.round(resultado.mean(),2)}%')

 Coeficiente de determinação médio : 0.83%


In [22]:
x_scaler = StandardScaler()
x_treino_scaler = x_scaler.fit_transform(x_treino)

In [23]:
y_scaler = StandardScaler()
y_treino_scaler = y_scaler.fit_transform(y_treino.reshape(-1,1))

In [24]:
y_teste_scaler = y_scaler.transform(y_teste.reshape(-1,1))

In [25]:
x_teste_scaler = x_scaler.transform(x_teste)

In [26]:
rede_scaler = MLPRegressor(hidden_layer_sizes=(50),activation='tanh',solver='adam',
             max_iter=1200,tol=0.0001, random_state=10)

#activation = relu,tanh,logistic, identity
#solver = adam,sgd,lbfgs

In [27]:
rede_scaler.fit(x_treino_scaler,y_treino_scaler.ravel())

MLPRegressor(activation='tanh', hidden_layer_sizes=50, max_iter=1200,
             random_state=10)

In [28]:
rede_scaler.score(x_treino_scaler,y_treino_scaler.ravel())

0.8660614878120646

In [29]:
rede_scaler.score(x_teste_scaler,y_teste_scaler.ravel())

0.8207074582435291

In [30]:
x_treino_inverse = x_scaler.inverse_transform(x_treino_scaler)
y_treino_inverse = y_scaler.inverse_transform(y_treino_scaler)
x_teste_inverse = x_scaler.inverse_transform(x_teste_scaler)
y_teste_inverse = y_scaler.inverse_transform(y_teste_scaler)

In [31]:
kfold = KFold(n_splits=15, shuffle=True, random_state=10)
modelo = rede_scaler
resultado = cross_val_score(modelo, independente,dependente, cv=kfold)
print(f' Coeficiente de determinação médio : {np.round(resultado.mean(),2)}%')

 Coeficiente de determinação médio : -1.17%


In [32]:
print(f'R² do treino : {rede.score(x_treino,y_treino)}')
print(f'R² do teste : {rede.score(x_teste,y_teste)}')
print(f'R² do treino escalonado : {rede_scaler.score(x_treino_scaler,y_treino_scaler.ravel())}')
print(f'R² do teste escalonado : {rede_scaler.score(x_teste_scaler,y_teste_scaler.ravel())}')

R² do treino : 0.849099752384026
R² do teste : 0.8175508861902884
R² do treino escalonado : 0.8660614878120646
R² do teste escalonado : 0.8207074582435291
